In [187]:
import pandas as pd
import numpy as np
import json
from urllib.parse import urlencode
import requests
import clickhouse_connect

In [188]:
with open('/home/jupyter-vladperesad/keys/clickhouse_connect_key.json') as src:
    data = json.load(src)

In [189]:
client = clickhouse_connect.get_client(host=data['host'],
                                       port=8443,
                                       username=data['user'],
                                       password=data['password'])

In [232]:
df_14 = client.query_df('''
SELECT
     at.student_id AS student_id,
     st.student_name AS student_name,
     gr.group_name as group_name,
     date,
     homework,
     behaviour,
     comprehension,
     vocabulary,
     speaking,
     reading,
     writing
FROM
    luxinedu_2.attendance AS at 
JOIN luxinedu_2.students AS st ON at.student_id=st.student_id
JOIN luxinedu_2.groups AS gr ON st.group_id=gr.group_id
WHERE date > now() - INTERVAL 14 DAY AND gr.group_id = 1
ORDER BY 
at.student_id,
date
''')

In [225]:
#create a subset that only contains student_ids
df_st = df_14['student_id']

In [226]:
#calculate percent difference in students performance beween most recent class and the class before that
df_pct = df_14 \
    .drop(columns=['student_name',
                    'group_name',
                    'date']) \
    .groupby('student_id') \
    .pct_change(fill_method=None)*100

In [227]:
df_pct = df_pct \
    .round(1) \
    .dropna(axis=0,
            how='all') \
    .join(df_st,
          how='inner')

In [233]:
df_7 = client.query_df('''
SELECT
     at.student_id AS student_id,
     st.student_name AS student_name,
     gr.group_name as group_name,
     date,
     homework,
     behaviour,
     comprehension,
     vocabulary,
     speaking,
     reading,
     writing
FROM
    luxinedu_2.attendance AS at 
JOIN luxinedu_2.students AS st ON at.student_id=st.student_id
JOIN luxinedu_2.groups AS gr ON st.group_id=gr.group_id
WHERE date > now() - INTERVAL 7 DAY AND gr.group_id = 1
ORDER BY 
at.student_id,
date
''')

In [230]:
final_table = df_7 \
    .merge(df_pct,
           how='left',
           on='student_id',
           suffixes=('_act','_pct'))

In [221]:
#in order to keep the token hidden it was saved in JSON file and can be read from it:
with open('/home/jupyter-vladperesad/keys/token_wed_1905.json') as src:
    data = json.load(src)

In [222]:
token = data['token']
chat_id = data['chat_id']

In [223]:
for index, row in final_table.iterrows():
    date = row['date']
    student_name = row['student_name']
    homework_a = row['homework_act']
    homework_p = row['homework_pct']
    behaviour_a = row['behaviour_act']
    behaviour_p = row['behaviour_pct']
    comprehension_a = row['comprehension_act']
    comprehension_p = row['comprehension_pct']
    vocabulary_a = row['vocabulary_act']
    vocabulary_p = row['vocabulary_pct']
    speaking_a = row['speaking_act']
    speaking_p = row['speaking_pct']
    reading_a = row['reading_act']
    reading_p = row['reading_pct']
    writing_a = row['writing_act']
    writing_p = row['writing_pct']
        
    message = f''' Report on the class on {date}
    Student : {student_name}
    Homework: {homework_a} ({homework_p}%)
    Behaviour: {behaviour_a} ({behaviour_p}%)
    Comprehension: {comprehension_a} ({comprehension_p}%)
    Vocabulary: {vocabulary_a} ({vocabulary_p}%)
    Speaking: {speaking_a} ({speaking_p}%)
    Reading: {reading_a} ({reading_p}%)
    Writing: {writing_a} ({writing_p}%)'''

    message = message
    params = {'chat_id': chat_id,
              'text': message}
    base_url = f'https://api.telegram.org/bot{token}/'
    url = base_url + 'sendMessage?' + urlencode(params)
    resp = requests.get(url)